# INF8775 – Analyse et conception d’algorithmes
# TP3 – Automne 2025

NOM, Prénom, 1234567

NOM, Prénom, 1234567

Note finale:

<u>**Date limite de remise :**</u>  6 décembre pour tous

# Instructions

## Rédaction et remise du rapport

- Ce notebook constitue à la fois le sujet du TP, votre code et votre rapport. Il contient déjà du code pour faciliter vos mesures et l'affichage de vos résultats, ainsi qu'un squelette pour votre rapport.

- Complétez directement le notebook, vous êtes libres de créer des nouvelles cellules de code ou de texte.

- Vous pouvez utiliser des fichiers externes pour stocker des exemplaires et des résultats, mais nous devons être capable de comprendre facilement votre démarche et de la reproduire.

- <u>**IMPORTANT**</u> Remettez le fichier du notebook sur Moodle avec le nom `MATRICULE1_MATRICULE2.ipynb`

- Vous pouvez inclure du code trouvé sur Internet, mais vous devez en mentionner la source, sous peine d'être sanctionnés pour plagiat.

## Mise en situation

Ce travail pratique se répartit sur deux séances de laboratoire et est une occasion de mettre en application les connaissances vues en cours. Vous devrez développer l'algorithme de votre choix pour essayer de résoudre le plus efficacement possible le problème donné. Une partie de la note sera accordée en fonction des résultats que vous obtiendrez par rapport aux autres équipes.

## Description du problème

Dans le cadre de ce TP, nous vous demandons de répondre à un problème combinatoire inspiré du puzzle [Selective Sponges](https://cacm.acm.org/opinion/upstart-puzzles-selective-sponges/) de Dennis E. Shasha (Communications of the ACM).
Le but est de construire un carré rempli de symboles de manière à satisfaire certaines contraintes sur les séquences qu’il contient.

On considère un carré de taille n × n à remplir à l’aide d’un vocabulaire de k symboles.
Certaines séquences de symboles sont bannies, d’autres sont objectives.
Chaque séquence objective présente dans le carré rapporte 1 point, et chaque séquence bannie enlève 1 point.

Votre objectif : concevoir un algorithme qui construit un carré maximisant le score total,
défini comme la différence entre le nombre de séquences objectifs et le nombre de séquences bannies présentes.

## Jeu de données

Dans ce TP, vous avez accès à beaucoup plus de paramètres que les autres fois. Ceux-ci sont : la taille du carré, la taille du vocabulaire, et le nombre de séquences objectifs et le nombre de séquences bannies. Pour générer de sexemplaires, utilisez la fonction generate_dataset qui prend une liste de tuple qui correspond aux quatre parmètres.

*Faites attentions à considérer différentes combinaisons de valeurs pour chaque paramètres autant pour la compétition que pour vos analyses.*


In [1]:
import random
import time
import statistics
from typing import List, Tuple, Dict


class SpongeGridProblem:
    """
    Représente une instance du problème.

    - n: dimension du carré n×n à produire
    - alphabet: liste de symboles permis (ex: ["A","B","C","I"])
    - targets: séquences qu'on veut VOIR dans la grille
    - banned: séquences qu'on veut ÉVITER dans la grille
    """
    def __init__(
        self,
        n: int,
        alphabet: List[str],
        targets: List[str],
        banned: List[str]
    ):
        self.n = n
        self.alphabet = alphabet
        self.targets = targets
        self.banned = banned


def random_word(alphabet: List[str], length: int) -> str:
    return "".join(random.choice(alphabet) for _ in range(length))


def generate_grid_instance(
    n: int,
    nb_targets: int,
    nb_banned: int,
    alphabet_size: int,
    min_len: int = 2,
    max_len: int = None,
    seed: int = None
) -> SpongeGridProblem:
    """
    Génére une instance avec:
    - un alphabet de taille 'taille_vocabulaire'
    - nb_targets séquences objectifs
    - nb_bannies séquences bannies
    Les longueurs des séquences sont entre min_len et max_len (par défaut max_len = n).
    """
    if seed is not None:
        random.seed(seed)

    if max_len is None:
        max_len = n

    # on construit un alphabet simple comme ['A','B','C','D', ...]
    base_letters = [chr(ord('A') + i) for i in range(alphabet_size)]
    alphabet = base_letters

    targets = [
        random_word(alphabet, random.randint(min_len, max_len))
        for _ in range(nb_targets)
    ]
    banned = [
        random_word(alphabet, random.randint(min_len, max_len))
        for _ in range(nb_banned)
    ]

    return SpongeGridProblem(
        n=n,
        alphabet=alphabet,
        targets=targets,
        banned=banned
    )


def sequences_in_line(line: str, seq: str) -> bool:
    """True si 'seq' apparaît comme sous-chaîne contiguë dans 'line'."""
    return seq in line


def count_matches_in_grid(grid: List[List[str]], seqs: List[str]) -> int:
    """
    Combien de séquences DISTINCTES dans `seqs` apparaissent au moins une fois
    dans le carré (en ligne gauche->droite ou en colonne haut->bas) ?
    """
    n = len(grid)
    # Prépare toutes les lignes (strings)
    rows = ["".join(grid[i][j] for j in range(n)) for i in range(n)]
    cols = ["".join(grid[i][j] for i in range(n)) for j in range(n)]

    found = 0
    for s in seqs:
        present = any(sequences_in_line(r, s) for r in rows) \
                  or any(sequences_in_line(c, s) for c in cols)
        if present:
            found += 1
    return found


def score_grid(
    grid: List[List[str]],
    problem: SpongeGridProblem,
    alpha: float = 1.0,
    beta: float = 1.0
) -> Dict[str, float]:
    """
    Calcule le score = alpha * (#objectifs couverts) - beta * (#bannies déclenchées)
    et renvoie aussi les sous-métriques.
    """
    covered_obj = count_matches_in_grid(grid, problem.targets)
    triggered_bad = count_matches_in_grid(grid, problem.banned)
    score_val = alpha * covered_obj - beta * triggered_bad
    return {
        "score": score_val,
        "covered": covered_obj,
        "triggered": triggered_bad
    }

def generate_samples(
    n: int,
    nb_targets: int,
    nb_banned: int,
    alphabet_size: int,
    num_samples: int = 5,
    seed_base: int = 0
) -> List[SpongeGridProblem]:
    """
    Génère plusieurs instances aléatoires qui partagent les mêmes paramètres
    (n, nb_targets, nb_banned, alphabet_size)
    pour permettre de moyenner la perf de l'algo sur cette famille.
    """
    samples = []
    for k in range(num_samples):
        pb = generate_grid_instance(
            n=n,
            nb_targets=nb_targets,
            nb_banned=nb_banned,
            alphabet_size=alphabet_size,
            seed=seed_base + k
        )
        samples.append(pb)
    return samples


def generate_dataset(
    param_list: List[Tuple[int, int, int, int]],
    num_samples: int = 5,
    seed_base: int = 0
) -> List[Tuple[Dict[str, int], List[SpongeGridProblem]]]:
    """
    Génère un dataset complet à partir d'une liste de combinaisons de paramètres.

    Args:
        param_list: Liste de tuples (n, nb_targets, nb_banned, alphabet_size).
                   Exemple: [
                       (5, 3, 2, 3),
                       (10, 5, 4, 4),
                       (15, 7, 6, 5)
                   ]
        num_samples: Nombre d'échantillons à générer pour chaque combinaison
        seed_base: Seed pour la génération aléatoire

    Returns:
        dataset : liste de samples
        - samples: liste des SpongeGridProblem générés
    """
    dataset = []

    for i, param_tuple in enumerate(param_list):
        n, nb_targets, nb_banned, alphabet_size = param_tuple


        samples = generate_samples(
            n=n,
            nb_targets=nb_targets,
            nb_banned=nb_banned,
            alphabet_size=alphabet_size,
            num_samples=num_samples,
            seed_base=seed_base + i * num_samples
        )

        dataset.append(samples)

    return dataset



In [11]:
# Exemple d'utilisation

banned = ["AB", "BA"]
targets = ["AA", "BB"]
alphabet = ["A", "B"]
n = 5

problem = SpongeGridProblem(n=n, alphabet=alphabet, targets=targets, banned=banned)

sample_grid = [
	['A', 'A', 'B', 'A', 'A'],
	['B', 'B', 'A', 'B', 'B'],
	['A', 'A', 'A', 'A', 'A'],
	['B', 'A', 'B', 'A', 'B'],
	['A', 'B', 'A', 'B', 'A']
]

result = score_grid(sample_grid, problem, alpha=1.0, beta=1.0)
print("Score:", result)


Score: {'score': 0.0, 'covered': 2, 'triggered': 2}


# Implantations et expérimentations

Ces fonctions auxiliaires vous sont fournies pour vérifier l'exactitude des vos algorithmes, mesurer leurs performance et afficher vos résultats.

Il est recommandé de prendre le temps de lire et comprendre le code.

Exécutez la cellule ci-dessous pour pouvoir utiliser les fonctions auxiliaires.

In [3]:
import time
from typing import Callable, List, Tuple, Dict
import matplotlib.pyplot as plt
import statistics as stats


class InvalidSolution(Exception):
    def __init__(self, msg="Invalid solution, verify your code."):
        super().__init__(msg)


class Measure:
    """Container for aggregated results over plusieurs échantillons d'une même configuration."""
    def __init__(
        self,
        n: int,
        alphabet_size: int,
        nb_targets: int,
        nb_banned: int,
        mean_time_ms: float,
        mean_score: float,
        mean_targets_covered: float,
        mean_banned_triggered: float,
    ) -> None:
        self.n = n
        self.alphabet_size = alphabet_size
        self.nb_targets = nb_targets
        self.nb_banned = nb_banned
        self.mean_time_ms = mean_time_ms
        self.mean_score = mean_score
        self.mean_targets_covered = mean_targets_covered
        self.mean_banned_triggered = mean_banned_triggered


def is_valid_grid(problem: SpongeGridProblem, grid: List[List[str]]) -> bool:
    """
    Valide la sortie de l'algo:
    - grid doit être une matrice n×n
    - chaque case doit être un symbole autorisé
    """
    # correct shape?
    if len(grid) != problem.n:
        return False
    for row in grid:
        if len(row) != problem.n:
            return False

    # allowed alphabet?
    allowed = set(problem.alphabet)
    for i in range(problem.n):
        for j in range(problem.n):
            if grid[i][j] not in allowed:
                return False

    return True


def measure_single_run(
    procedure: Callable[[SpongeGridProblem], List[List[str]]],
    problem: SpongeGridProblem,
    alpha: float = 1.0,
    beta: float = 1.0,
    time_scale: int = 1000
) -> Tuple[int, Dict[str, float]]:
    """
    Exécute une fois l'algo sur un seul problème.

    Retourne un tuple (elapsed_ms, metrics_dict)
    où metrics_dict contient:
        {
          "score": float,
          "covered": int,
          "triggered": int
        }

    Exceptions:
        InvalidSolution: si la grille retournée est invalide
    """
    start = time.time() * time_scale
    grid = procedure(problem)
    end = time.time() * time_scale

    # structural validation
    if not is_valid_grid(problem, grid):
        raise InvalidSolution("Invalid grid shape or symbols.")

    # compute score
    metrics = score_grid(grid, problem, alpha=alpha, beta=beta)
    # metrics = {"score":..., "covered":..., "triggered":...}

    return (round(end - start), metrics)





def measure_mean(
procedure: Callable[[SpongeGridProblem], List[List[str]]],
    problems: List[SpongeGridProblem],
    alpha: float = 1.0,
    beta: float = 1.0,
    time_scale: int = 1000
) -> Measure:
    """
    1. Génère num_samples instances avec les mêmes paramètres.
    2. Lance l'algo sur chacune.
    3. Calcule les moyennes.

    Peut lever InvalidSolution si UNE des exécutions retourne une grille invalide.
    """

    times = []
    scores = []
    covered_list = []
    triggered_list = []

    for pb in problems:
        elapsed_ms, metrics = measure_single_run(
            procedure,
            pb,
            alpha=alpha,
            beta=beta,
            time_scale=time_scale
        )
        times.append(elapsed_ms)
        scores.append(metrics["score"])
        covered_list.append(metrics["covered"])
        triggered_list.append(metrics["triggered"])

    mean_time_ms = sum(times) / len(times)
    mean_score = sum(scores) / len(scores)
    mean_cov = sum(covered_list) / len(covered_list)
    mean_trig = sum(triggered_list) / len(triggered_list)

    return Measure(
        n=problems[0].n,
        alphabet_size=len(problems[0].alphabet),
        nb_targets=len(problems[0].targets),
        nb_banned=len(problems[0].banned),
        mean_time_ms=mean_time_ms,
        mean_score=mean_score,
        mean_targets_covered=mean_cov,
        mean_banned_triggered=mean_trig,
    )


def measure_range(
    procedure: Callable[[SpongeGridProblem], List[List[str]]],
    problems_dataset : List[List[SpongeGridProblem]],
    alpha: float = 1.0,
    beta: float = 1.0,
    time_scale: int = 1000
) -> List[Measure]:
    """
    Applique measure_mean_for_spec() pour chaque config dans specs.
    Retourne une liste de Measure.
    """
    measures: List[Measure] = []
    for problems in problems_dataset:
        m = measure_mean(
            procedure=procedure,
            problems=problems,
            alpha=alpha,
            beta=beta,
            time_scale=time_scale
        )
        measures.append(m)
    return measures


def display_data_as_table(measures: List[Measure]):
    """Affiche un tableau récapitulatif des moyennes mesurées."""
    header = "{: <20} {: <20} {: <20} {: <20} {: <20} {: <20} {: <20} {: <20}"
    row    = "{: <20} {: <20} {: <20} {: <20} {: <20} {: <20} {: <20} {: <20}"

    print(
        header.format(
            "Taille du carré",
            "Taille de l'alphabet",
            "Nombre d'objectifs",
            "Nombre de bannis",
            "Temps(ms)",
            "Score",
            "Objectifs atteints",
            "Pénalités accrus"
        )
    )
    for m in measures:
        print(
            row.format(
                m.n,
                m.alphabet_size,
                m.nb_targets,
                m.nb_banned,
                m.mean_time_ms,
                m.mean_score,
                m.mean_targets_covered,
                m.mean_banned_triggered
            )
        )


### The different tests are below, the names are in french to avoid confusion

def display_test_puissance(vals, title="Test de puissance"):
    x = list(vals.keys())
    y = list(vals.values())

    # Perform linear regression
    m, b = stats.linear_regression(x, y)

    r = list(map(lambda x : m*x + b, x))

    # Create the line equation
    line_eq = f"y = {m:.2f}x + {b:.2f}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, r, color="red", label='Regression linéaire')

    # Add labels and title
    plt.xlabel('log Taille')
    plt.ylabel('log Temps')
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

def display_test_rapport(vals, title="Test du rapport"):
    x = list(vals.keys())
    y = list(vals.values())

    plt.plot(x, y, label='Mesures')
    plt.scatter(x, y, label='Mesures')

    # Add labels and title
    plt.xlabel('Taille')
    plt.ylabel('Temps / f(taille)')
    plt.title(title)
    plt.show()


def display_test_constantes(vals, title="Test des constantes"):
    x = list(vals.keys())
    y = list(vals.values())

    # Perform linear regression
    m, b = stats.linear_regression(x, y)

    r = list(map(lambda x : m*x + b, x))

    # Create the line equation
    line_eq = f"y = {m:.2E}x + {b:.2E}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, r, color="red", label='Regression linéaire')

    # Add labels and title
    plt.xlabel('f(Taille)')
    plt.ylabel('Temps')
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')


    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()


# Algorithme (5 pts)

Votre algorithme sera en partie noté en fonction d'une évaluation relative entre les équipes. 4 points seront donnés aux équipes qui se classeront dans le premier quartile lors de notre évaluation sur un ensemble d'exemplaires. Les équipes se trouvant dans le quartile dont les algorithmes ont le moins bien performé recevront 1 point.

**IMPORTANT** Votre algo doit retourner une solution après 3 minutes. Si ce n'est pas le cas, vous serez pénalisé.

In [4]:
def algo(problem: SpongeGridProblem) -> List[List[str]]:
	n = problem.n
	solution = [[random.choice(problem.alphabet) for _ in range(n)] for _ in range(n)]
	return solution


In [7]:
specs = [
    (4, 6, 6, 4),
    (6, 10, 8, 4),
    (8, 15, 15, 5),
] # Modifiable -- liste de (n, nb_targets, nb_banned, alphabet_size) pour évaluer la performance

dataset=generate_dataset(specs)

results = measure_range(
    procedure=algo,      # votre algo(problem) -> grid n×n
    problems_dataset=dataset,
)



In [6]:
display_data_as_table(results)

Taille du carré      Taille de l'alphabet Nombre d'objectifs   Nombre de bannis     Temps(ms)            Score                Objectifs atteints   Pénalités accrus    
4                    4                    6                    6                    0.2                  0.2                  2.0                  1.8                 
6                    4                    10                   8                    0.0                  1.4                  3.0                  1.6                 
8                    5                    15                   15                   0.0                  -0.6                 3.0                  3.6                 


# Analyse asymptotique (2 pts)

Effectuer une analyse asymptotique de votre algorithme.

# Analyse hybride (6 pts)

Effectuer une analyse hybride de votre algorithme.

# Analyse code carbon (1 pts)

Effectuer une anlayse code carbon en sélectionnant différent pays pour l'analyse. Commenter vos résultats.

## Évolution de votre algorithme et Conclusion (6 pts)

Faites une synthèse de vos analyses pour mettre en évidence les qualités et défauts de votre algorithme. Mentionnez les améliorations qui vous ont permis d'atteindre votre algorithme actuel ainsi que des pistes d'améliorations potentielles restantes.

 ## Autres critères
 Qualité du code

Présentation générale

- Concision
- Qualité du français

Pénalité retard
- -2 pt / journée de retard, arrondi vers le haut. Les TPs ne sont plus acceptés après 3 jours.